In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Orçamento\\ONE DRIVE ORCAMENTO\\OneDrive - GRUPO RETEC\\02. Engenharia\\IA\\ORCA\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\Orçamento\\ONE DRIVE ORCAMENTO\\OneDrive - GRUPO RETEC\\02. Engenharia\\IA\\ORCA'

# === ENTITY ===

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    learning_rate: float
    n_estimators: float
    target_column: float

# === CONFIGURATION MANAGER ===

In [8]:
from ORCA.constants import *
from ORCA.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBoosting
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            learning_rate = params.learning_rate,
            n_estimators = params.n_estimators,
            target_column = schema.name
            
        )

        return model_trainer_config

# === COMPONENT ===

In [13]:
import pandas as pd
import os
from ORCA import logger
from sklearn.linear_model import ElasticNet
import joblib

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
from statsmodels.stats.outliers_influence import variance_inflation_factor

from statsmodels.tools import add_constant
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.model_selection import cross_val_score
import joblib
import optuna

c:\Users\Orçamento\ONE DRIVE ORCAMENTO\OneDrive - GRUPO RETEC\02. Engenharia\IA\ORCA\.orcaenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_excel(self.config.train_data_path)
        test_data = pd.read_excel(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))



<>:11: SyntaxWarning: invalid escape sequence '\('
<>:12: SyntaxWarning: invalid escape sequence '\('
<>:11: SyntaxWarning: invalid escape sequence '\('
<>:12: SyntaxWarning: invalid escape sequence '\('
C:\Users\Orçamento\AppData\Local\Temp\ipykernel_33316\152597204.py:11: SyntaxWarning: invalid escape sequence '\('
  train_data.columns = train_data.columns.str.replace("PreÃ§o\(1\)", "Preço (1)", regex=True)
C:\Users\Orçamento\AppData\Local\Temp\ipykernel_33316\152597204.py:12: SyntaxWarning: invalid escape sequence '\('
  test_data.columns = test_data.columns.str.replace("PreÃ§o\(1\)", "Preço (1)", regex=True)


In [ ]:
print(train_data.columns)

In [19]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-07-25 15:50:11,652: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-25 15:50:11,660: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-25 15:50:11,668: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-25 15:50:11,671: INFO: common: created directory at: artifacts]
[2025-07-25 15:50:11,675: INFO: common: created directory at: artifacts/model_trainer]


C:\Users\Orçamento\AppData\Local\Temp\ipykernel_33316\2411560366.py:10: SyntaxWarning: invalid escape sequence '\('
  train_data.columns = train_data.columns.str.replace("PreÃ§o\(1\)", "Preço (1)", regex=True)
C:\Users\Orçamento\AppData\Local\Temp\ipykernel_33316\2411560366.py:11: SyntaxWarning: invalid escape sequence '\('
  test_data.columns = test_data.columns.str.replace("PreÃ§o\(1\)", "Preço (1)", regex=True)


KeyError: "['PreÃ§o(1)'] not found in axis"